In [32]:
from __future__ import division
import numpy as np
from numpy.random import rand
import matplotlib.pyplot as plt
import csv
import pandas as pd
from tqdm import tnrange, tqdm
from datetime import datetime
import xlwt
import multiprocessing
from xlwt import Workbook
import json
from collections import defaultdict
get_ipython().run_line_magic('matplotlib', 'notebook')
import matplotlib.pyplot as plt

class Ising():
    ''' Simulating the Ising model '''    
    ## monte carlo moves
    def mcmove(self, config, N, beta, h):
        ''' This is to execute the monte carlo moves using 
        Metropolis algorithm such that detailed
        balance condition is satisified'''
        for i in range(N):
            for j in range(N):
                for k in range(N):
                    a = np.random.randint(1, N-1)
                    b = np.random.randint(0, N)
                    c = np.random.randint(0, N)
                    if a == len(config) :
                        s =  config[a, b, c]
                        nb = config[(a+1)%N,b,c] + config[a,(b+1)%N,c] + config[(a-1)%N,b,c] + config[a,(b-1)%N,c] + config[a,b,(c+1)%N] + config[a,b,(c-1)%N] - h 
                        cost = 2*s*nb
                    elif a == 0:
                        s =  config[a, b, c]
                        nb = config[(a+1)%N,b,c] + config[a,(b+1)%N,c] + config[(a-1)%N,b,c] + config[a,(b-1)%N,c] + config[a,b,(c+1)%N] + config[a,b,(c-1)%N] + h 
                        cost = 2*s*nb
                    else :
                        s =  config[a, b, c]
                        nb = config[(a+1)%N,b,c] + config[a,(b+1)%N,c] + config[(a-1)%N,b,c] + config[a,(b-1)%N,c] + config[a,b,(c+1)%N] + config[a,b,(c-1)%N]
                        cost = 2*s*nb
                    if cost < 0:	
                        s *= -1
                    elif rand() < np.exp(-1*cost*beta):
                        s *= -1
                    else:
                        s *= 1
                    config[a, b, c] = s
        return config  
    
    def configPlot(self, f, config, i, N, n_):
        ''' This modules plts the configuration once passed to it along with time etc '''
        X, Y = np.meshgrid(range(N+1), range(N+1))
        sp =  f.add_subplot(1, 1, n_ )  
        plt.setp(sp.get_yticklabels(), visible=False)
        plt.setp(sp.get_xticklabels(), visible=False)      
        plt.pcolormesh(X, Y, config, cmap=plt.cm.RdBu);
        plt.title('Time=%d'%i); plt.axis('tight')    
    plt.show()
    
    def calcEnergy(self, config, h):
        '''Energy of a given configuration'''
        N = len(config)
        energy = 0
        for i in range(N):
            for j in range(N):
                for k in range(N):
                    if i == N:
                        S = config[i,j,k]
                        config[(a+1)%N,b,c] + config[a,(b+1)%N,c] + config[(a-1)%N,b,c] + config[a,(b-1)%N,c] + config[a,b,(c+1)%N] + config[a,b,(c-1)%N]
                        nb = config[(i+1)%N, j, k] + config[i,(j+1)%N, k] + config[(i-1)%N, j, k] + config[i,(j-1)%N, k] + config[i,j,(k+1)%N] + config[i,j,(k-1)%N] + h
                        energy += -nb*S
                    elif i == 0:
                        S = config[i,j,k]
                        nb = config[(i+1)%N, j, k] + config[i,(j+1)%N, k] + config[(i-1)%N, j, k] + config[i,(j-1)%N, k] + config[i,j,(k+1)%N] + config[i,j,(k-1)%N] - h
                        energy += -nb*S
                    else :
                        S = config[i,j,k]
                        nb = config[(i+1)%N, j, k] + config[i,(j+1)%N, k] + config[(i-1)%N, j, k] + config[i,(j-1)%N, k] + config[i,j,(k+1)%N] + config[i,j,(k-1)%N] 
                        energy += -nb*S
        return energy/6.


    def calcMag(self, config):
        '''Magnetization of a given configuration'''
        mag = np.sum(config)
        return mag
    
    def InteractionTop(self, config, N):
        '''Interaction energy of the top surface'''
        count=[[0]*N]*N
        for j in range(N):
            for k in range(N):
                count[j,k] = config[N-1,j,k]*config[N-2,j,k]
        count1 = np.array(count)    
            return (-1*count1)

    def InteractionBottom(self, config, N):
        '''Interaction energy of the bottom surface'''
        count=[[0]*N]*N
        for j in range(N):
            for k in range(N):
                count[j,k] = config[0,j,k]*config[1,j,k]
        count1 = np.array(count)    
            return (-1*count1)
    
    def CorrelationTop(self, Top, N, i, j):
        Correlation_Parallel = []
        Correlation_Diagonal = []
        for k in range(N/2):
            Correlation_Parallel.append((Top[i+k,j] + Top[i-k,j] + Top[i,j+k] + Top[i,j-k])/4)
            Correlation_Diagonal.append((Top[i+k,j+k] + Top[i-k,j+k] + Top[i-k,j+k] + Top[i-k,j-k])/4)
        return(Correlation_Diagonal, Correlation_Parallel)
    
    def CorrelationTop(self, Bottom, N, i, j):
        Correlation_Parallel = []
        Correlation_Diagonal = []
        for k in range(N/2):
            Correlation_Parallel.append((Bottom[i+k,j] + Bottom[i-k,j] + Bottom[i,j+k] + Bottom[i,j-k])/4)
            Correlation_Diagonal.append((Bottom[i+k,j+k] + Bottom[i-k,j+k] + Bottom[i-k,j+k] + Bottom[i-k,j-k])/4)
        return(Correlation_Diagonal, Correlation_Parallel)
    
    def average(self, a):
        count = 0
        for i in range(len(a)):
            count += a[i]
        count /= len(a)
        return(count)

    def convert_np(data):
        final_data=defaultdict(dict)
        for k1,v1 in data.items():
            for k2,v2 in v1.items():
                temp=[]
                for each in v2:
                    temp.append(np.asarray(each))    
                final_data[int(k1)][int(k2)]=tuple(temp)
            
        return final_data  
            
    
    with open('C:/Users/DEV KAKKAD/Desktop/Academics/SEM 7/Project - MonteCarlo/data.json') as f:
        points_data = json.load(f, parse_int=int)        
            
    points_data=convert_np(points_data)
    
    def get_xy():
        for density, scales_data in points_data.items():
            for scale,corros_points in scales_data.items():
                for each_line in corros_points:
                    for x in each_line[0]:
                        for y in each_line[1]:
                            pass
                        print(str(x)+", "+str(y)) 
                    
    with open('C:/Users/DEV KAKKAD/Desktop/Academics/SEM 7/Project - MonteCarlo/data.json') as f:
        points_data = json.load(f, parse_int=int)  
    
    densities=list(points_data.keys())

    densities    
    
    def simulate(self):   
        
        ''' This module simulates the Ising model'''
        
        '''Defining all parameters'''
        
        N = 4      # Number of Rows
        h = 0        #external field
        N_intermediate_points = 1       #  number of temperature points
        T_min = 1.40                         #  lower range for temperature
        T_max = 2.10                         #  upper range for temperature
        #T = np.linspace(T_min, T_max, N_intermediate_points);      #divisions T min and T max into different temperature steps 
        T = 1.8
        msrmnt = 10001
        DeltaPsi1 = []
        DeltaPsi2 = []
        DeltaPsi3 = []
        DeltaPsi4 = []
        Magnetization = [[0]*1000]*1000
        Energy = []
        iteration = []
        Correlation_Parallel = []
        Correlation_Diagonal = []
        for i in range(N):
            if i%100 == 0:
                iteration.append(i)
     
        #----------------------------------------------------------------------------------
        
        '''generating the solid-liquid system'''
        
        config = 2*np.random.rand(N,N,N)-1
        
        for i in range(N):
            for j in range(N):
                for k in range(N):
                    if i!=0 and i!=N-1:
                        config[i,j] = 1.0
        
        #print(config)
        
        '''code for heterogeneous surface'''
        
        with open('C:/Users/DEV KAKKAD/Desktop/Academics/SEM 7/Project - MonteCarlo/data.json') as f:
            points_data = json.load(f, parse_int=int)
        
        for each_line in points_data['64032']['1']:
            print(each_line)
            for x in each_line[0]:
                for y in each_line[1]:
                    config[0,x,y] = -0.1
                    config[N-1,x,y] = -0.1
            

            
        '''Plotting the solid surface for visualisation'''
        
        surface = 2*np.random.rand(N,N)-1
        for i in range(N):
            for j in range(N):
                surface[i,j] = config[0,i,j]
        print(surface)
        
        f = plt.figure(figsize=(15, 15), dpi=80);
        
        self.configPlot(f, surface, 0, N, 1);

        '''Data Calculation'''
        
        for i in range(msrmnt):
            self.mcmove(config, N, 1.0/T, h)
                
            if i%100 == 0:
                DeltaPsi1.append((np.sum(self.InteractionBottom(config, N)) + np.sum(self.InteractionTop(config, N)))/2)
                Energy.append(self.calcEnergy(config, h))
                
                for j in range(N):
                    Magnetisation[j,int(i//100)] = np.sum(config[j,:,:])
        
        psi1 = self.average(DeltaPsi1)
        for i in range(len(DeltaPsi1)):
            DeltaPsi2.append((DeltaPsi1[i]-psi1)**2)
            DeltaPsi3.append((DeltaPsi1[i]-psi1)**3)
            DeltaPsi4.append((DeltaPsi1[i]-psi1)**4)
            
        psi2 = self.average(DeltaPsi2)
        psi3 = self.average(DeltaPsi3)
        psi4 = self.average(DeltaPsi4)
        
        print(round(psi1,4), round(psi2, 4), round(psi3, 4), round(psi4, 4))
        
        f = plt.figure(figsize=(9, 5)); # plot the calculated values

        plt.scatter(iteration, Energy, s=50, marker='o', color='IndianRed')
        plt.axis([T_min-0.2, T_max+0.2, -0.0023, 0])
        plt.xlabel("Iteraion", fontsize=20);
        plt.ylabel("Energy", fontsize=20);         plt.axis('tight');
        
        Bottom = self.InteractionBottom(config, N)
        Top = self.InteractionTop(config, N)
        
        


In [ ]:
import multiprocessing

rm = Ising

processes = []

for _ in range(10):
    p = multiprocessing.Process(target=rm.simulate)
    p.start
    processes.append(p)
    
for process in processes:
    process.join()
